In [9]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from flask import Flask, render_template, redirect
import requests
import pymongo

In [10]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\radgu\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [11]:
# URL of the page to be scrapped
url = 'https://redplanetscience.com/'
browser.visit(url)

In [12]:
 # HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve the latest news title
news_title=soup.find_all('div', class_='content_title')[0].text

# Retrieve the latest news paragraph
news_p=soup.find_all('div', class_='rollover_description_inner')[0].text

news_title
news_p

IndexError: list index out of range

In [4]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

NameError: name 'pymongo' is not defined

In [5]:
# Define database and collection
db = client.commerce_db
collection = db.items

NameError: name 'client' is not defined

In [5]:
# URL of page to be scraped
url = 'https://redplanetscience.com/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [5]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all('div', class_='caption')

# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = result.find('a', class_='title').text
        # Identify and return price of listing
        price = result.find('h4', class_='price').text
        # Identify and return link to listing
        link = result.a['href']

        # Run only if title, price, and link are available
        if (title and price and link):
            # Print results
            print('-------------')
            print(title)
            print(price)
            print(link)

            # Dictionary to be inserted as a MongoDB document
            post = {
                'title': title,
                'price': price,
                'url': link
            }

            collection.insert_one(post)

    except Exception as e:
        print(e)

-------------
Asus VivoBook X4...
$295.99
/test-sites/e-commerce/allinone/product/545
-------------
Prestigio SmartB...
$299.00
/test-sites/e-commerce/allinone/product/546
-------------
Prestigio SmartB...
$299.00
/test-sites/e-commerce/allinone/product/547
-------------
Aspire E1-510
$306.99
/test-sites/e-commerce/allinone/product/517
-------------
Lenovo V110-15IA...
$321.94
/test-sites/e-commerce/allinone/product/548
-------------
Lenovo V110-15IA...
$356.49
/test-sites/e-commerce/allinone/product/549
-------------
Hewlett Packard...
$364.46
/test-sites/e-commerce/allinone/product/550
-------------
Acer Aspire 3 A3...
$372.70
/test-sites/e-commerce/allinone/product/551
-------------
Acer Aspire A315...
$379.94
/test-sites/e-commerce/allinone/product/552
-------------
Acer Aspire ES1-...
$379.95
/test-sites/e-commerce/allinone/product/553
-------------
Acer Aspire 3 A3...
$391.48
/test-sites/e-commerce/allinone/product/554
-------------
Acer Aspire 3 A3...
$393.88
/test-sites/e-comme

In [6]:
# Display items in MongoDB collection
listings = db.items.find()

for listing in listings:
    print(listing)

{'_id': ObjectId('623d0aae143d57c00602d7b5'), 'title': 'Asus VivoBook X4...', 'price': '$295.99', 'url': '/test-sites/e-commerce/allinone/product/545'}
{'_id': ObjectId('623d0aae143d57c00602d7b6'), 'title': 'Prestigio SmartB...', 'price': '$299.00', 'url': '/test-sites/e-commerce/allinone/product/546'}
{'_id': ObjectId('623d0aae143d57c00602d7b7'), 'title': 'Prestigio SmartB...', 'price': '$299.00', 'url': '/test-sites/e-commerce/allinone/product/547'}
{'_id': ObjectId('623d0aae143d57c00602d7b8'), 'title': 'Aspire E1-510', 'price': '$306.99', 'url': '/test-sites/e-commerce/allinone/product/517'}
{'_id': ObjectId('623d0aae143d57c00602d7b9'), 'title': 'Lenovo V110-15IA...', 'price': '$321.94', 'url': '/test-sites/e-commerce/allinone/product/548'}
{'_id': ObjectId('623d0aae143d57c00602d7ba'), 'title': 'Lenovo V110-15IA...', 'price': '$356.49', 'url': '/test-sites/e-commerce/allinone/product/549'}
{'_id': ObjectId('623d0aae143d57c00602d7bb'), 'title': 'Hewlett Packard...', 'price': '$364.46